# Neural networks code and tests notebook

# This notebook takes the flux, photon index and fratio data and try to predict classes from this data on a :

# - LSTM network
# - ConvLSTM network
# - Conv1D network

In [14]:
import numpy as np
from numpy import random
import pandas as pd
import re
import os
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import math
from scipy import interpolate
import sys 
from re import search
from astropy.io import fits
from astropy import units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord
from astropy.coordinates import ICRS, Galactic, FK4, FK5  # Low-level frames
from astropy.coordinates import Angle, Latitude, Longitude
import shutil
from keras.callbacks import CSVLogger
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import glob
from datetime import datetime
from keras.layers import Conv1D, Dense, Dropout, Input, Concatenate, GlobalMaxPooling1D, BatchNormalization
from keras.layers.convolutional import MaxPooling1D
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from matplotlib import pyplot
import tensorflow.keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten,LSTM
from keras.utils import to_categorical
from keras.layers import Dense, Activation, Concatenate
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit # or StratifiedShuffleSplit
from pandas import DataFrame
from pyts.classification import BOSSVS
from pyts.multivariate.classification import MultivariateClassifier
import shutil
from keras.callbacks import CSVLogger
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit # or StratifiedShuffleSplit
from pandas import DataFrame
import sktime
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier

In [15]:
pathON="C:/Users/pierr/Documents_kanop/Shps/Pierre_points_for_louis/Bureau/interpo_ONOFF_GOLD/ON_data/"
pathOFF="C:/Users/pierr/Documents_kanop/Shps/Pierre_points_for_louis/Bureau/interpo_ONOFF_GOLD/OFF_data/"

In [16]:
#Storing in a list the adresses of all the data files in the notebook

data_files_ON=[]
file_names_ON=[]

data_files_OFF=[]
file_names_OFF=[]
data_files_ALL=[]
file_names_ALL=[]

#Storing in a list the adresses of all the data files in the notebook
for filename in os.listdir(pathOFF):
    f = os.path.join(pathOFF,filename)
    if os.path.isfile(f):
        data_files_OFF.append(f)
        file_names_OFF.append(filename)
        data_files_ALL.append(f)
        file_names_ALL.append(filename)
for filename in os.listdir(pathON):
    f = os.path.join(pathON,filename)
    if os.path.isfile(f):
        data_files_ON.append(f)
        file_names_ON.append(filename)
        data_files_ALL.append(f)
        file_names_ALL.append(filename)        

In [17]:
#Using data_files with only the 36 binning
Filter=True
binning=37


if Filter==True:
    
    idx_OFF=[]
    idx_ON=[]
    dataON=[]
    dataOFF=[]
    for i in range(len(data_files_OFF)):
        dataframe = pd.read_csv(data_files_OFF[i])
        lg = len(dataframe)
        
        if lg==binning:
            idx_OFF.append(i)
    
    for i in range(len(data_files_ON)):
        dataframe = pd.read_csv(data_files_ON[i])
        lg = len(dataframe)
        if lg==binning:
            idx_ON.append(i)


    for i in range(len(idx_OFF)):

        a=idx_OFF[i]
        dataOFF.append(data_files_OFF[a])
    for i in range(len(idx_ON)):

        a=idx_ON[i]
        dataON.append(data_files_ON[a])

idx = idx_OFF+idx_ON


In [18]:


lgON=len(dataON)
lgOFF=len(dataOFF)
lg=lgON+lgOFF
Labels = []

nbfeatures=2

a=binning
b=nbfeatures
c=lg

# multivariate

data_matrix= np.zeros((b,c,a))

#Construct data matrix
for j in range(len(dataOFF)):

    dataframe=pd.read_csv(dataOFF[j],index_col=[0])
    dataframe.columns=['MJD','Flux','Delta Flux','Photon Index','Delta Index','TS','fratio']

    data_matrix[0][j]=dataframe['Flux']
    data_matrix[1][j]=dataframe['Photon Index']
    
for j in range(len(dataON)):

    v=j+len(dataOFF)
    dataframe=pd.read_csv(dataON[j],index_col=[0])
    dataframe.columns=['MJD','Flux','Delta Flux','Photon Index','Delta Index','TS','fratio']

    data_matrix[0][v]=dataframe['Flux']
    data_matrix[1][v]=dataframe['Photon Index']
    
#Creating labels
for i in range(lgOFF):
    
    Labels.append(int(0))
    
for i in range(lgON):
    Labels.append(int(1))



In [19]:
#Reshaping data matrix
data_matrix=data_matrix.reshape(c,nbfeatures,binning)

In [ ]:
#LSTM architecture
from keras.layers import Conv1D
from sklearn.metrics import f1_score
ON_accuracy=[]
OFF_accuracy=[]
fscore=[]
FPOS=[]
FNEG= []

weight_for_0 = (1 / lgOFF) * (lg / 2.0)
weight_for_1 = (1 / lgON) * (lg / 2.0)

class_weight = {0: weight_for_0, 1: weight_for_1}
callback=tensorflow.keras.callbacks.EarlyStopping(monitor='binary_accuracy',patience=2, verbose=0, mode='auto',baseline=None, restore_best_weights=True)




for i in range(300):
    
    model = Sequential()
    model.add(LSTM(9,input_shape=(nbfeatures,binning),activation='tanh'))
    model.add(Dense(2, activation='softmax'))
    model.compile(loss=tensorflow.keras.losses.BinaryCrossentropy(reduction='sum'),optimizer='SGD', metrics=['binary_accuracy'])
    
    x_train, x_test, y_train, y_test = train_test_split(data_matrix, Labels, test_size=0.2, random_state=i)
    y_test2=y_test.copy()
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)
    
    #fitting data
    model.fit(x_train, y_train,epochs=5,batch_size=1,class_weight=class_weight,callbacks=callback)

    print('Iteration no.',i)
    #Obtain the accuracy of prediction for each class
    prediction= model.predict(x_test)
    predicted_labels=[]
    
    for i in range(len(prediction)):
        if prediction[i][0]>0.5:
            predicted_labels.append(0)
        else:
            predicted_labels.append(1)
    on_score=0
    on_nbs=0
    off_nbs=0
    off_score=0
    foff=0
    fon=0
    for i in range(len(y_test)):
        if y_test2[i]==1 :
            on_nbs+=1
            if predicted_labels[i]==1:
                on_score+=1
            else : 
                fon+=1
        if y_test2[i]==0 :
            off_nbs+=1
            if predicted_labels[i]==0:
                off_score+=1 
            else:
                foff+=1
    if on_nbs>0:    
        ON_accuracy.append(100*(on_score/on_nbs))
        FPOS.append(100*(fon/on_nbs))
    OFF_accuracy.append(100*(off_score/off_nbs))
    FNEG.append(100*(foff/off_nbs))
    f1= f1_score(y_test2,predicted_labels,average='weighted')
    fscore.append(f1)
print("Accuracy for ON class: ",np.mean(ON_accuracy) ,"%")
print("Accuracy for OFF class: ",np.mean(OFF_accuracy) ,"%")
print("False Positive rate: ",np.mean(FNEG) ,"%")
print("False Negative rate: ",np.mean(FPOS) ,"%")
print("F1 score: ",np.mean(fscore))

Epoch 1/5
871/871 [==============================] - 3s 2ms/step - loss: 0.6434 - binary_accuracy: 0.5700
Epoch 2/5
871/871 [==============================] - 1s 2ms/step - loss: 0.5774 - binary_accuracy: 0.5568
Epoch 3/5
871/871 [==============================] - 1s 2ms/step - loss: 0.5816 - binary_accuracy: 0.5844
Epoch 4/5
871/871 [==============================] - 1s 2ms/step - loss: 0.5436 - binary_accuracy: 0.5568
Epoch 5/5
871/871 [==============================] - 1s 2ms/step - loss: 0.5125 - binary_accuracy: 0.5580
Iteration no. 0
7/7 [==============================] - 1s 2ms/step
Epoch 1/5
871/871 [==============================] - 3s 2ms/step - loss: 0.6543 - binary_accuracy: 0.5253
Epoch 2/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5908 - binary_accuracy: 0.5224
Epoch 3/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5604 - binary_accuracy: 0.5235
Iteration no. 1
7/7 [==============================] - 1s 2ms/step
Epoch 1/5
871/871 

871/871 [==============================] - 4s 2ms/step - loss: 0.6299 - binary_accuracy: 0.5212
Epoch 2/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5718 - binary_accuracy: 0.5626
Epoch 3/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5250 - binary_accuracy: 0.6889
Epoch 4/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5351 - binary_accuracy: 0.5913
Epoch 5/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5245 - binary_accuracy: 0.5591
Iteration no. 32
7/7 [==============================] - 1s 1ms/step
Epoch 1/5
871/871 [==============================] - 4s 2ms/step - loss: 0.6439 - binary_accuracy: 0.5109
Epoch 2/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5866 - binary_accuracy: 0.5235
Epoch 3/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5594 - binary_accuracy: 0.5258
Epoch 4/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5369 - binary_accur

871/871 [==============================] - 2s 2ms/step - loss: 0.5241 - binary_accuracy: 0.5763
Iteration no. 64
7/7 [==============================] - 1s 2ms/step
Epoch 1/5
871/871 [==============================] - 4s 2ms/step - loss: 0.6335 - binary_accuracy: 0.4885
Epoch 2/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5776 - binary_accuracy: 0.5166
Epoch 3/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5459 - binary_accuracy: 0.5189
Epoch 4/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5224 - binary_accuracy: 0.5339
Epoch 5/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5142 - binary_accuracy: 0.5855
Iteration no. 65
7/7 [==============================] - 0s 2ms/step
Epoch 1/5
871/871 [==============================] - 4s 2ms/step - loss: 0.5823 - binary_accuracy: 0.5264
Epoch 2/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5465 - binary_accuracy: 0.6016
Epoch 3/5
871/871 [=======

7/7 [==============================] - 0s 2ms/step
Epoch 1/5
871/871 [==============================] - 4s 2ms/step - loss: 0.6564 - binary_accuracy: 0.4506
Epoch 2/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5963 - binary_accuracy: 0.5258
Epoch 3/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5607 - binary_accuracy: 0.5258
Epoch 4/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5365 - binary_accuracy: 0.5258
Iteration no. 81
7/7 [==============================] - 1s 2ms/step
Epoch 1/5
871/871 [==============================] - 4s 2ms/step - loss: 0.6685 - binary_accuracy: 0.3858
Epoch 2/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5966 - binary_accuracy: 0.5385
Epoch 3/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5669 - binary_accuracy: 0.5235
Epoch 4/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5382 - binary_accuracy: 0.5247
Iteration no. 82
7/7 [===========

Epoch 1/5
871/871 [==============================] - 4s 2ms/step - loss: 0.6416 - binary_accuracy: 0.5379
Epoch 2/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5736 - binary_accuracy: 0.5936
Epoch 3/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5333 - binary_accuracy: 0.5327
Epoch 4/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5193 - binary_accuracy: 0.5396
Iteration no. 115
7/7 [==============================] - 1s 2ms/step
Epoch 1/5
871/871 [==============================] - 4s 2ms/step - loss: 0.6596 - binary_accuracy: 0.5052
Epoch 2/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5983 - binary_accuracy: 0.5316
Epoch 3/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5613 - binary_accuracy: 0.5327
Epoch 4/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5433 - binary_accuracy: 0.5316
Epoch 5/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5278 - b

871/871 [==============================] - 1s 2ms/step - loss: 0.5997 - binary_accuracy: 0.5465
Epoch 3/5
871/871 [==============================] - 1s 2ms/step - loss: 0.5531 - binary_accuracy: 0.5304
Iteration no. 146
7/7 [==============================] - 0s 1ms/step
Epoch 1/5
871/871 [==============================] - 3s 2ms/step - loss: 0.6576 - binary_accuracy: 0.5534
Epoch 2/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5667 - binary_accuracy: 0.5281
Epoch 3/5
871/871 [==============================] - 2s 2ms/step - loss: 0.5308 - binary_accuracy: 0.5293
Iteration no. 147
7/7 [==============================] - 0s 2ms/step
Epoch 1/5
871/871 [==============================] - 3s 2ms/step - loss: 0.6753 - binary_accuracy: 0.4575
Epoch 2/5
871/871 [==============================] - 1s 2ms/step - loss: 0.5895 - binary_accuracy: 0.5304
Epoch 3/5
871/871 [==============================] - 1s 2ms/step - loss: 0.5584 - binary_accuracy: 0.5304
Epoch 4/5
871/871 [=====

In [22]:
# Conv/LSTM architecture
#LSTM architecture
from keras.layers import Conv1D
from sklearn.metrics import f1_score
ON_accuracy=[]
OFF_accuracy=[]
fscore=[]
FPOS=[]
FNEG= []

weight_for_0 = (1 / lgOFF) * (lg / 2.0)
weight_for_1 = (1 / lgON) * (lg / 2.0)

class_weight = {0: weight_for_0, 1: weight_for_1}
callback=tensorflow.keras.callbacks.EarlyStopping(monitor='binary_accuracy',patience=3, verbose=0, mode='auto',baseline=None, restore_best_weights=True)




for i in range(300):
    
    model = Sequential()
    model.add(Dense(binning, activation='tanh', input_shape=(nbfeatures,binning),input_dim=nbfeatures))
    model.add(Conv1D(filters=20, kernel_size=1, activation='tanh'))
    model.add(Dropout(0.2))
    model.add(LSTM(20,input_shape=(20, nbfeatures), activation='tanh'))
    model.add(Flatten())
    model.add(Dense(2, activation='softmax'))
    model.compile(loss=tensorflow.keras.losses.BinaryCrossentropy(reduction='sum'),optimizer='SGD', metrics=['binary_accuracy'])
    x_train, x_test, y_train, y_test = train_test_split(data_matrix, Labels, test_size=0.2, random_state=i)
    y_test2=y_test.copy()
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)
    
    #fitting data
    model.fit(x_train, y_train,epochs=5,batch_size=1,class_weight=class_weight,callbacks=callback)

    print('Iteration no.',i)
    #Obtain the accuracy of prediction for each class
    prediction= model.predict(x_test)
    predicted_labels=[]
    
    for i in range(len(prediction)):
        if prediction[i][0]>0.5:
            predicted_labels.append(0)
        else:
            predicted_labels.append(1)
    on_score=0
    on_nbs=0
    off_nbs=0
    off_score=0
    foff=0
    fon=0
    for i in range(len(y_test)):
        if y_test2[i]==1 :
            on_nbs+=1
            if predicted_labels[i]==1:
                on_score+=1
            else : 
                fon+=1
        if y_test2[i]==0 :
            off_nbs+=1
            if predicted_labels[i]==0:
                off_score+=1 
            else:
                foff+=1
    if on_nbs>0:    
        ON_accuracy.append(100*(on_score/on_nbs))
        FPOS.append(100*(fon/on_nbs))
    OFF_accuracy.append(100*(off_score/off_nbs))
    FNEG.append(100*(foff/off_nbs))
    f1= f1_score(y_test2,predicted_labels,average='weighted')
    fscore.append(f1)
print("Accuracy for ON class: ",np.mean(ON_accuracy) ,"%")
print("Accuracy for OFF class: ",np.mean(OFF_accuracy) ,"%")
print("False Positive rate: ",np.mean(FNEG) ,"%")
print("False Negative rate: ",np.mean(FPOS) ,"%")
print("F1 score: ",np.mean(fscore))

Epoch 1/15
871/871 [==============================] - 4s 2ms/step - loss: 0.7026 - binary_accuracy: 0.3513
Epoch 2/15
871/871 [==============================] - 2s 2ms/step - loss: 0.6082 - binary_accuracy: 0.7153
Epoch 3/15
871/871 [==============================] - 2s 2ms/step - loss: 0.5608 - binary_accuracy: 0.6131
Epoch 4/15
871/871 [==============================] - 2s 2ms/step - loss: 0.5242 - binary_accuracy: 0.6556
Epoch 5/15
871/871 [==============================] - 2s 2ms/step - loss: 0.4694 - binary_accuracy: 0.6877
Iteration no. 0
7/7 [==============================] - 1s 2ms/step
Epoch 1/15
871/871 [==============================] - 4s 3ms/step - loss: 0.7055 - binary_accuracy: 0.3330
Epoch 2/15
871/871 [==============================] - 2s 2ms/step - loss: 0.6698 - binary_accuracy: 0.5293
Epoch 3/15
871/871 [==============================] - 2s 2ms/step - loss: 0.5926 - binary_accuracy: 0.5706
Epoch 4/15
871/871 [==============================] - 2s 2ms/step - loss: 0.5

In [23]:
from sklearn.metrics import f1_score

ON_accuracy=[]
OFF_accuracy=[]
fscore=[]
FPOS=[]
FNEG=[]
weight_for_0 = (1 / lgOFF) * (lg / 2.0)
weight_for_1 = (1 / lgON) * (lg / 2.0)

class_weight = {0: weight_for_0, 1: weight_for_1}
#Measuring the meaned accuracy of  correct prediction of labellisation for each class  over a defined number of steps
for i in range(300):
    nb_filters=20
    
    callback=tensorflow.keras.callbacks.EarlyStopping(monitor='binary_accuracy',patience=2, verbose=0, mode='auto',baseline=None, restore_best_weights=True)

    #Conv1D Architecture
    model = Sequential()
    model.add(Conv1D(nb_filters, 1, padding="same", activation="relu",input_shape=(nbfeatures,binning)))
    model.add(Conv1D(15, 1, padding="same", activation="relu",input_shape=(nb_filters,binning)))
    model.add(Conv1D(9, 1, padding="same", activation="relu",input_shape=(15,binning)))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(9, activation='relu'))
    model.add(Dense(2, activation='softmax'))
    model.compile(loss=tensorflow.keras.losses.BinaryCrossentropy(reduction='sum'),optimizer='SGD', metrics=['binary_accuracy'])


    x_train, x_test, y_train, y_test = train_test_split(data_matrix, Labels, test_size=0.2, random_state=i)
    y_test2=y_test.copy()
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)
    
    #fitting data
    model.fit(x_train, y_train,epochs=4,batch_size=1,class_weight=class_weight,callbacks=callback)

    print('Iteration no.',i)
    #Obtain the accuracy of prediction for each class
    prediction= model.predict(x_test)
    predicted_labels=[]
    
    for i in range(len(prediction)):
        if prediction[i][0]>0.5:
            predicted_labels.append(0)
        else:
            predicted_labels.append(1)
    on_score=0
    on_nbs=0
    off_nbs=0
    off_score=0
    foff=0
    fon=0
    for i in range(len(y_test)):
        if y_test2[i]==1 :
            on_nbs+=1
            if predicted_labels[i]==1:
                on_score+=1
            else : 
                fon+=1
        if y_test2[i]==0 :
            off_nbs+=1
            if predicted_labels[i]==0:
                off_score+=1 
            else:
                foff+=1
    if on_nbs>0:    
        ON_accuracy.append(100*(on_score/on_nbs))
        FPOS.append(100*(fon/on_nbs))
    OFF_accuracy.append(100*(off_score/off_nbs))
    FNEG.append(100*(foff/off_nbs))
    f1= f1_score(y_test2,predicted_labels,average='weighted')
    fscore.append(f1)
print("Accuracy for ON class: ",np.mean(ON_accuracy) ,"%")
print("Accuracy for OFF class: ",np.mean(OFF_accuracy) ,"%")
print("False Positive rate: ",np.mean(FNEG) ,"%")
print("False Negative rate: ",np.mean(FPOS) ,"%")
print("F1 score: ",np.mean(fscore))

Epoch 1/4
871/871 [==============================] - 2s 2ms/step - loss: 0.7084 - binary_accuracy: 0.3249
Epoch 2/4
871/871 [==============================] - 2s 2ms/step - loss: 0.6728 - binary_accuracy: 0.5534
Epoch 3/4
871/871 [==============================] - 2s 2ms/step - loss: 0.6181 - binary_accuracy: 0.6613
Epoch 4/4
871/871 [==============================] - 1s 2ms/step - loss: 0.5645 - binary_accuracy: 0.6900
Iteration no. 0
7/7 [==============================] - 0s 2ms/step
Epoch 1/4
871/871 [==============================] - 2s 2ms/step - loss: 0.6921 - binary_accuracy: 0.3800
Epoch 2/4
871/871 [==============================] - 1s 2ms/step - loss: 0.6545 - binary_accuracy: 0.5947
Epoch 3/4
871/871 [==============================] - 1s 2ms/step - loss: 0.6440 - binary_accuracy: 0.5947
Epoch 4/4
871/871 [==============================] - 2s 2ms/step - loss: 0.5995 - binary_accuracy: 0.5959
Iteration no. 1
7/7 [==============================] - 0s 2ms/step
Epoch 1/4
871/871 